In [51]:
# https://wikidocs.net/64765

# 03. 단어 단위 RNN - 임베딩 사용

### 03. 단어 단위 RNN - 임베딩 사용
---

In [52]:
import torch
import torch.nn as nn
import torch.optim as optim

In [53]:
# 단어 집합 만들기
sentence = "Repeat is the best medicine for memory".split()

vocab = list(set(sentence))
print(vocab)

['medicine', 'best', 'for', 'memory', 'the', 'is', 'Repeat']


In [54]:
# 단어에 고유한 정수 부여
word2index = {tkn: i for i, tkn in enumerate(vocab, 1)}  # 단어에 고유한 정수 부여
word2index['<unk>']=0
print(word2index)

{'medicine': 1, 'best': 2, 'for': 3, 'memory': 4, 'the': 5, 'is': 6, 'Repeat': 7, '<unk>': 0}


In [55]:
# 단어에 부여된 정수 보기
print(word2index['memory'])

4


In [56]:
# 데이터를 단어로 바꾸는 함수

# 수치화된 데이터를 단어로 바꾸기 위한 사전
index2word = {v: k for k, v in word2index.items()}
print(index2word)

{1: 'medicine', 2: 'best', 3: 'for', 4: 'memory', 5: 'the', 6: 'is', 7: 'Repeat', 0: '<unk>'}


In [57]:
print(index2word[2])

best


In [58]:
# 데이터 생성 함수
def build_data(sentence, word2index):
    encoded = [word2index[token] for token in sentence] # 각 문자를 정수로 변환. 
    input_seq, label_seq = encoded[:-1], encoded[1:] # 입력 시퀀스와 레이블 시퀀스를 분리
    input_seq = torch.LongTensor(input_seq).unsqueeze(0) # 배치 차원 추가
    label_seq = torch.LongTensor(label_seq).unsqueeze(0) # 배치 차원 추가
    return input_seq, label_seq

X, Y = build_data(sentence, word2index)

In [59]:
# 데이터 확인
print(X)
print(Y)

tensor([[7, 6, 5, 2, 1, 3]])
tensor([[6, 5, 2, 1, 3, 4]])


In [60]:
# 모델 생성 클래스 만들기

class Net(nn.Module):
    def __init__(self, vocab_size, input_size, hidden_size, batch_first=True):
        super(Net, self).__init__()
        self.embedding_layer = nn.Embedding(num_embeddings=vocab_size, # 워드 임베딩
                                            embedding_dim=input_size)
        self.rnn_layer = nn.RNN(input_size, hidden_size, # 입력 차원, 은닉 상태의 크기 정의
                                batch_first=batch_first)
        self.linear = nn.Linear(hidden_size, vocab_size) # 출력은 원-핫 벡터의 크기를 가져야함.
                                                         # 또는 단어 집합의 크기만큼 가져야함.

    def forward(self, x):
        # 1. 임베딩 층
        # 크기변화: (배치 크기, 시퀀스 길이) => (배치 크기, 시퀀스 길이, 임베딩 차원)
        output = self.embedding_layer(x)
        # 2. RNN 층
        # 크기변화: (배치 크기, 시퀀스 길이, 임베딩 차원)
        # => output (배치 크기, 시퀀스 길이, 은닉층 크기), hidden (1, 배치 크기, 은닉층 크기)
        output, hidden = self.rnn_layer(output)
        # 3. 최종 출력층
        # 크기변화: (배치 크기, 시퀀스 길이, 은닉층 크기) => (배치 크기, 시퀀스 길이, 단어장 크기)
        output = self.linear(output)
        # 4. view를 통해서 배치 차원 제거
        # 크기변화: (배치 크기, 시퀀스 길이, 단어장 크기) => (배치 크기*시퀀스 길이, 단어장 크기)
        return output.view(-1, output.size(2))

In [61]:
# 하이퍼 파라미터
vocab_size = len(word2index)  # 단어장의 크기는 임베딩 층, 최종 출력층에 사용된다. <unk> 토큰을 크기에 포함한다.
input_size = 5  # 임베딩 된 차원의 크기 및 RNN 층 입력 차원의 크기
hidden_size = 20  # RNN의 은닉층 크기

In [62]:
# 모델 생성
model = Net(vocab_size, input_size, hidden_size, batch_first=True)
# 손실함수 정의
loss_function = nn.CrossEntropyLoss() # 소프트맥스 함수 포함이며 실제값은 원-핫 인코딩 안 해도 됨.
# 옵티마이저 정의
optimizer = optim.Adam(params=model.parameters())

In [63]:
# 임의로 예측해보기. 가중치는 전부 랜덤 초기화 된 상태이다.
output = model(X)
print(output)

tensor([[-0.0072,  0.0741, -0.0157, -0.1446,  0.2672,  0.3108,  0.0604,  0.1494],
        [-0.1916,  0.2689, -0.1323, -0.2928,  0.2735,  0.0943,  0.0610, -0.0803],
        [-0.0912,  0.2004, -0.0017, -0.1631,  0.1500, -0.1098,  0.0476,  0.2324],
        [-0.0623,  0.3361,  0.0684, -0.2861,  0.2314,  0.0676,  0.1911,  0.2721],
        [ 0.0836,  0.3116, -0.1217, -0.1499,  0.2918,  0.0693,  0.0598,  0.0543],
        [-0.0671,  0.2613, -0.0327, -0.1346,  0.1930, -0.0554, -0.0244,  0.0417]],
       grad_fn=<ViewBackward>)


In [64]:
print(output.shape)

torch.Size([6, 8])


In [65]:
# decoding 함수 만들기

# 수치화된 데이터를 단어로 전환하는 함수
decode = lambda y: [index2word.get(x) for x in y]

In [67]:
# 200 epoch 학습

# 훈련 시작
for step in range(201):
    # 경사 초기화
    optimizer.zero_grad()
    # 순방향 전파
    output = model(X)
    # 손실값 계산
    loss = loss_function(output, Y.view(-1))
    # 역방향 전파
    loss.backward()
    # 매개변수 업데이트
    optimizer.step()
    # 기록
    if step % 40 == 0:
        print("[{:02d}/201] {:.4f} ".format(step+1, loss))
        pred = output.softmax(-1).argmax(-1).tolist()
        print(" ".join(["Repeat"] + decode(pred)))
        print()

[01/201] 2.0568 
Repeat the memory Repeat medicine medicine medicine

[41/201] 1.5694 
Repeat is the best medicine for memory

[81/201] 0.9102 
Repeat is the best medicine for memory

[121/201] 0.4334 
Repeat is the best medicine for memory

[161/201] 0.2123 
Repeat is the best medicine for memory

[201/201] 0.1195 
Repeat is the best medicine for memory

